In [1]:
# %load_ext signature
%matplotlib inline
import os

import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd

In [2]:
import json
from bokeh.io import show, output_notebook
from bokeh.models import (CDSView, ColorBar, ColumnDataSource,
                          CustomJS, CustomJSFilter, 
                          GeoJSONDataSource, HoverTool,
                          LinearColorMapper, Slider)
from bokeh.layouts import column, row, widgetbox
from bokeh.palettes import brewer
from bokeh.plotting import figure

## LET'S READ THE SHAPEFILE OF THE NETHERLANDS

In [3]:
shapefile = os.path.expanduser('~/Netherlands/Source/Shapefile/Gemeenten.shp')
gdf = gpd.read_file(shapefile)
gdf.sample(5)

,OBJECTID,Gemeenteco,Gemeentena,Shape__Are,Shape__Len,geometry
256,257,GM0851,Steenbergen,1.511742e+08,55508.635232,"POLYGON ((4.36826 51.65842, 4.37253 51.65539, ..."
65,66,GM1598,Koggenland,8.331989e+07,47597.741150,"POLYGON ((4.87650 52.64097, 4.87541 52.64232, ..."
89,90,GM1696,Wijdemeren,7.636090e+07,49568.164590,"POLYGON ((5.05689 52.23654, 5.05628 52.23713, ..."
5,6,GM0327,Leusden,5.889249e+07,42722.740845,"POLYGON ((5.41567 52.15044, 5.41610 52.15073, ..."
25,26,GM0225,Druten,4.245985e+07,37702.850862,"POLYGON ((5.61397 51.89930, 5.62811 51.89940, ..."


### Just play with some columns

In [4]:
gdf1 = gdf[['Gemeentena', 'geometry']].to_crs('EPSG:28992') # to Amersfoort/RD New
# gdf1.plot(figsize=figsize)

In [5]:
gdf1.sort_values('Gemeentena', ignore_index=True, inplace=True)
gdf1.head()

,Gemeentena,geometry
0,'s-Gravenhage,"MULTIPOLYGON (((80751.323 460570.933, 82569.45..."
1,'s-Hertogenbosch,"POLYGON ((148377.215 409562.923, 148377.510 40..."
2,Aa en Hunze,"POLYGON ((239628.716 560674.968, 239339.409 56..."
3,Aalsmeer,"POLYGON ((111204.432 476147.215, 111283.021 47..."
4,Aalten,"POLYGON ((229935.834 435391.008, 229870.536 43..."


## COVID DATA - 11/04/2020

In [6]:
covid_df = pd.read_csv('Covid_data/covid_NL_Apr11.csv', delimiter=';')
covid_df.head()

,Category,Zkh opname per 100.000,Zkh opname,Meldingen per 100.000,Meldingen
0,'s-Hertogenbosch,66.4,103,172.1,267
1,Aa en Hunze,3.9,1,31.4,8
2,Aalsmeer,22.0,7,84.8,27
3,Aalten,3.7,1,77.4,21
4,Achtkarspelen,0.0,0,18.0,5


### Combine shapefile with covid data

In [7]:
covid_nl = gdf1.merge(covid_df, left_on='Gemeentena', right_on='Category')
covid_nl.head()

,Gemeentena,geometry,Category,Zkh opname per 100.000,Zkh opname,Meldingen per 100.000,Meldingen
0,'s-Hertogenbosch,"POLYGON ((148377.215 409562.923, 148377.510 40...",'s-Hertogenbosch,66.4,103,172.1,267
1,Aa en Hunze,"POLYGON ((239628.716 560674.968, 239339.409 56...",Aa en Hunze,3.9,1,31.4,8
2,Aalsmeer,"POLYGON ((111204.432 476147.215, 111283.021 47...",Aalsmeer,22.0,7,84.8,27
3,Aalten,"POLYGON ((229935.834 435391.008, 229870.536 43...",Aalten,3.7,1,77.4,21
4,Achtkarspelen,"POLYGON ((201448.385 579310.771, 201384.888 57...",Achtkarspelen,0.0,0,18.0,5


convert the data to a format that is conducive to mapping

Ref https://towardsdatascience.com/walkthrough-mapping-basics-with-bokeh-and-geopandas-in-python-43f40aa5b7e9

In [8]:
# Input GeoJSON source that contains features for plotting
geosource = GeoJSONDataSource(geojson = covid_nl.to_json())

In [9]:
output_notebook() # Bokeh will plot in this notebook

Loading BokehJS ...

Now that the data has been converted to the appropriate format, we can plot the map of state populations using the following steps:

1. Create the figure object

2. Add a patch renderer to the figure

3. Create a hover tool

4. Show the figure

In [10]:
# Create figure object.
p = figure(title = 'Covid dashboard, Netherlands',
        #    plot_height = 800 , plot_width = 600,
           match_aspect=True,
           toolbar_location = 'below',
           tools = 'pan, wheel_zoom, box_zoom, reset')
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
p.axis.visible = False

#Define a sequential multi-hue color palette.
palette = brewer['YlGnBu'][7]
#Reverse color order so that dark blue is highest obesity.
palette = palette[::-1]
#Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors.
plot_column = 'Zkh opname per 100.000'
color_mapper = LinearColorMapper(palette = palette, low = 0, high = covid_nl['Zkh opname per 100.000'].max())

#Define custom tick labels for color bar.
# tick_labels = {'0': '0%', '5': '5%', '10':'10%', '15':'15%', '20':'20%', '25':'25%', '30':'30%','35':'35%', '40': '>40%'}
#Create color bar. 
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8,width = 500, height = 20,
border_line_color=None,location = (0,0), orientation = 'horizontal')
# border_line_color=None,location = (0,0), orientation = 'horizontal', major_label_overrides = tick_labels)

# Add patch renderer to figure.
states = p.patches('xs','ys', source = geosource,
                   fill_color = {'field' :'Zkh opname per 100.000', 'transform' : color_mapper}, # choose field/values to display
                   line_color = 'gray', 
                   line_width = 0.25, 
                   fill_alpha = 1)
#Specify figure layout.
p.add_layout(color_bar, 'below')

# Create hover tool
p.add_tools(HoverTool(renderers = [states],
                      tooltips = [('Municipality','@Gemeentena'), 
                                  ('Meldingen', '@Meldingen'),
                                  ('Zkh opname per 100.000', '@{Zkh opname per 100.000}')]))
show(p)